# Lesson-10 Movie comments classification

1. prepare the raw data
2. sentences vectorization
3. classify comments with Neural Network

# data prepare

In [10]:
import pandas as pd
import jieba
import re
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
file_path = '/home/student/project/project-01/nlp_bots/hw-xujing/movie_comments.csv'

In [12]:
data = pd.read_csv(file_path, encoding='utf-8')

/home/student/anaconda3/envs/nlp_bots/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
data.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [14]:
data.shape

(261497, 5)

In [15]:
len(data)

261497

In [16]:
#remove empty comments
stars = list(data['star'])
comments = list(data['comment'])
index = list(data['id'])

In [17]:
for i in range(len(stars)):
    if not isinstance(stars[i], str):
        stars[i] = str(stars[i])

In [18]:
remove1 = [_i for _i, com in enumerate(comments) if (not isinstance(com, str))
          or com == '' or len(com) <= 4]
remove2 = [_i for _i, star in enumerate(stars) if star not in ['1', '2', '3', '4', '5']]
print(len(remove1), len(remove2))
removes = remove1 + remove2
removes = set(removes)

22047 1


In [19]:
movie_stars = [star for _i, star in enumerate(stars) if _i not in removes]
movie_comments = [comm for _i, comm in enumerate(comments) if _i not in removes]

In [20]:
len(movie_stars), len(movie_comments)

(239449, 239449)

In [21]:
# traditional Chinese convert to simplified Chinese
from hanziconv import HanziConv
def tra_to_sim(line):
    trans = HanziConv.toSimplified(line)
    return trans


In [22]:
movie_comments = [tra_to_sim(comm) for comm in movie_comments]

In [23]:
# remove not Chinese comments
def is_chinese(string):
    length = 0
    for char in string:
        if char >= '\u4e00' and char <= '\u9fff':
            length += 1
    if length > len(string) * 0.3:
        return True
    return False
    

In [24]:
%time
for i in range(1000):
    if not is_chinese(movie_comments[i]):
        print(movie_comments[i])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
You got a dream, you gotta protect it. Don't let your dreams be dreams.
You have a dream, you got to protect it.
This is part of my life……
This is part of my life……
http://www.douban.com/review/1279547/
2008.2.3
Smith毁容鸟...
2008.2.3
Smith毁容鸟...
http://v.youku.com/v_show/id_XODY3ODI3Ng==.html
真尴尬啊。大结局居然是十年后问了声“my name is hanmeimei whats your name？”
真尴尬啊。大结局居然是十年后问了声“my name is hanmeimei whats your name？”
真尴尬啊。大结局居然是十年后问了声“my name is hanmeimei whats your name？”
我只认识sandy and sue，再见
ヽ(´o｀；
ヽ(´o｀；
ヽ(´o｀；
ヽ(´o｀；


In [25]:
removes = [i for i, com in enumerate(movie_comments) if not is_chinese(com)]


In [26]:
len(removes)

9120

In [27]:
clean_stars = [star for i, star in enumerate(movie_stars) if i not in removes]
clean_comments = [comm for i, comm in enumerate(movie_comments) if i not in removes]

In [28]:
len(clean_stars), len(clean_comments)

(230329, 230329)

1.classify different type according with stars
2-classifier, 1-2 stars together, 3-5 stars together

In [29]:
stars_dict = Counter(clean_stars)
stars_dict

Counter({'1': 21370, '2': 25038, '4': 74220, '5': 51425, '3': 58276})

In [30]:
#get data: class pairs
def get_classes(dataset, labels):
    datas = defaultdict(list)
    for i in range(len(labels)):
        datas[labels[i]].append(dataset[i])
    return datas
    

In [31]:
class_label_dict = get_classes(clean_comments, clean_stars)

In [32]:
class_label_dict.keys()

dict_keys(['1', '2', '4', '5', '3'])

# Neural Network

two-types classification as pos_comments for 4 to 5 stars, neg_comments for 1 to 2 stars, ignore 3-stars for now

In [33]:
data_dict = defaultdict(list)
data_dict[0] = class_label_dict['1'] + class_label_dict['2']
data_dict[1] = class_label_dict['4'] + class_label_dict['5']

In [34]:
len(data_dict[0]), len(data_dict[1])

(46408, 125645)

In [35]:
import random
# choose balanced negative and positive samples 
sample_numbers = 45000
def get_balanced_data(data, sample_numbers):
    dataset = []
    labels = []
    for key in data:
        if len(data[key]) < sample_numbers:
            dataset += data[key]
            labels += [key] * len(data[key])
        else:
            dataset += random.sample(data[key], sample_numbers)
            labels += [key] * sample_numbers
    return dataset, labels

In [36]:
dataset, labels = get_balanced_data(data_dict, sample_numbers)

In [37]:
len(dataset), len(labels)

(90000, 90000)

In [38]:
print(dataset[250], labels[250])

打擂台叫五毒教上啊 0


# TF-IDF documents vectorized

In [8]:
import re
pattern = re.compile(r'[\w, 。；：！（）《》、“”‘’……]+')

In [39]:
#with open('/home/student/project/project-01/nlp_bots/hw-xujing/comment.txt', 'w', encoding='utf-8') as f:
    comments_cut = []
    for da in dataset:
        if not da.strip(): continue
        comment = ''.join(pattern.findall(da))
        comment = ' '.join(jieba.cut(comment))
        if not comment: continue
        comments_cut.append(comment)
        #f.write(comment+'/n')
    

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.864 seconds.
Prefix dict has been built succesfully.


In [40]:
len(comments_cut)

90000

In [1]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
sentences = LineSentence('/home/student/project/project-01/nlp_bots/hw-xujing/comment.txt')
model = Word2Vec(sentences=sentences, size=300, min_count=2, workers=4)

In [2]:
model.save('/home/student/project/project-01/nlp_bots/hw-xujing/douban.model')

In [3]:
model = Word2Vec.load('/home/student/project/project-01/nlp_bots/hw-xujing/douban.model')

In [4]:
model.wv['好看'].shape

(300,)

In [5]:
len(model.wv.vocab)

50527

#training sentence vectors

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
tfidf = TfidfVectorizer(max_features=1500)
X = tfidf.fit_transform(comments_cut)

In [42]:
X.shape

(90000, 1500)

In [44]:
X[1]

<1x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [45]:
labels = np.array(labels)

In [46]:
from sklearn.model_selection import train_test_split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, labels, test_size=0.2)
valid_dataset, test_dataset, valid_labels, test_labels = train_test_split(test_dataset, test_labels, test_size=0.5)


In [47]:
print(train_dataset.shape, valid_dataset.shape, test_dataset.shape)

(72000, 1500) (9000, 1500) (9000, 1500)


In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [49]:
clf = RandomForestClassifier()
clf.fit(train_dataset, train_labels)
y_hat = clf.predict(valid_dataset)
acc = format(accuracy_score(valid_labels, y_hat), '.4f')
print(acc)

/home/student/anaconda3/envs/nlp_bots/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7512


In [50]:
# save classified data
# here save in metrics format for tfidf vectors
from six.moves import cPickle as pickle
pickle_file = '/home/student/project/project-01/nlp_bots/hw-xujing/douban-tfidf'
try:
    with open(pickle_file, 'wb') as f:
        save = {
            'train_dataset' : train_dataset,
            'train_labels' : train_labels,
            'valid_dataset' : valid_dataset,
            'valid_labels' : valid_labels, 
            'test_dataset' : test_dataset,
            'test_labels' : test_labels
        }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('unable to save data to', pickle_file, ':', e)
    

# Neural Network Model

In [51]:
from __future__ import print_function
import tensorflow as tf
from six.moves import cPickle as pickle

In [52]:
import os
pickle_file = '/home/student/project/project-01/nlp_bots/hw-xujing/douban-tfidf'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del(save)
    print('train set', train_dataset.shape, train_labels.shape)
    print('valid set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape, test_dataset.shape)

train set (72000, 1500) (72000,)
valid set (9000, 1500) (9000,)
test set (9000, 1500) (9000, 1500)


In [53]:
train_dataset[1]

<1x1500 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [54]:
#unify data classes, make labels to one-hot 
import numpy as np
num_labels = 2
def reformat(dataset, labels):
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    datasets = dataset.astype(np.float32)
    datasets = datasets.toarray()
    return datasets, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('train set', train_dataset.shape, train_labels.shape)
print('valid set', valid_dataset.shape, valid_labels.shape)
print('test set', test_dataset.shape, test_labels.shape)

train set (72000, 1500) (72000, 2)
valid set (9000, 1500) (9000, 2)
test set (9000, 1500) (9000, 2)


In [62]:
# two layer nn
batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 128
beta_regul = 1e-3
feature_size = 1500


graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, feature_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step =tf.Variable(0)
    
    #variables
    # layer 1: stddev is the pointed derived data standard mean 
    weights1 = tf.Variable(tf.truncated_normal([feature_size, num_hidden_nodes1], stddev=np.sqrt(2.0 / (feature_size))))
    bias1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    #layer 2: input num_hidden_nodes1, 
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], 
                                               stddev=np.sqrt(2.0 / (num_hidden_nodes1))))
    bias2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    #last layer, input num_hidden_nodes2, current nodes= num_labels
    weights3 = tf.Variable(tf.truncated_normal([num_hidden_nodes2, num_labels],
                                              stddev=np.sqrt(2.0 / (num_hidden_nodes2))))
    bias3 = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + bias1)
    lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) +bias2)
    logits = tf.matmul(lay2_train, weights3) +bias3
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=tf_train_labels, logits=logits))
    L2 = beta_regul * (tf.nn.l2_loss(weights1) +tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
    loss = tf.reduce_mean(loss + L2)
    
    #optimizer
    learning_rate = tf.train.exponential_decay(0.1, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #predictions for train valid test
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + bias1)
    lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) +bias2)
    valid_prediction = tf.nn.softmax(tf.matmul(lay2_valid, weights3) + bias3)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + bias1)
    lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + bias2)
    test_prediction = tf.nn.softmax(tf.matmul(lay2_test, weights3) + bias3)

In [66]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 3601
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # pick an offset within the training data, which has been randomized.
        #分母减去batch_size是防止当batch_size不能被训练数据量整除时，offset:(offset + batch_size)超出数组界限
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #generate a mini batch
        batch_data = train_dataset[offset:(offset +batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #prepare a dictionary telling the session where to feed teh minibatch.
        #the key of the dictionary is the placeholder to feed teh minibatch.
        #and teh value is teh numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('minibatch loss as step %d: %f' % (step, l))
            print('minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
minibatch loss as step 0: 1.585643
minibatch accuracy: 46.9%
Validation accuracy: 50.9%
minibatch loss as step 200: 1.537333
minibatch accuracy: 68.8%
Validation accuracy: 57.3%
minibatch loss as step 400: 1.485980
minibatch accuracy: 63.3%
Validation accuracy: 65.8%
minibatch loss as step 600: 1.397760
minibatch accuracy: 68.0%
Validation accuracy: 73.3%
minibatch loss as step 800: 1.308116
minibatch accuracy: 70.3%
Validation accuracy: 73.4%
minibatch loss as step 1000: 1.263986
minibatch accuracy: 69.5%
Validation accuracy: 75.5%
minibatch loss as step 1200: 1.259528
minibatch accuracy: 75.8%
Validation accuracy: 75.8%
minibatch loss as step 1400: 1.176658
minibatch accuracy: 80.5%
Validation accuracy: 75.6%
minibatch loss as step 1600: 1.215387
minibatch accuracy: 72.7%
Validation accuracy: 72.3%
minibatch loss as step 1800: 1.160054
minibatch accuracy: 76.6%
Validation accuracy: 73.7%
minibatch loss as step 2000: 1.169119
minibatch accuracy: 75.8%
Validation accuracy: 

TFIDF准确率76.4%，需要进一步处理数据，或Learning rate试一下